In [17]:
import seaborn as sns

from bot import Bot
from field import Field
from game import Game

bot = Bot()
moves_survived = []
for _ in range(100):
    game = Game(False, bot)
    game.play()
    print()
    print(len(game.spawned_atoms), game.field.atoms)
    moves_survived.append(len(game.spawned_atoms))

sns.histplot(moves_survived)

..

.......
173 [-1, -1, -1, -1, 3, -1, 5, -1, -1, 4, -1, -1, -1, 7, -1, 8, -1, 6, 2]
.............................
568 [-1, 4, -1, 7, 19, 19, 11, -1, 13, 19, 14, 17, 16, 16, 17, 20, 18, 15, 8]
..................

In [16]:
from pandas import DataFrame
import numpy as np



def preprocess_data(field, plus, minus, next, op, decision):
    # inputs:
    #   18 atoms on the board, times 10 for index (first if lowest, adding one until 8 + lowest (and anything higher), and then -1 (Plus))
    #   13 inputs for current atom (1-9 for element, 10 for -1, 11 for -2, 12 for -3, 13 for -4)
    #   1 input for guaranteed plus next turn
    #   1 input for guaranteed minus next turn
    #   1 input for ability to change to plus atom (in minus)
    # outputs:
    #   18 for on-board decision
    #   2 for op, to switch or not

    input_vector = np.zeros(18 * 10 + 3 + 13)

    min_element = (next if next > 0 else 1e9)
    for a in field:
        if (a > 0): min_element = min(min_element, a)
    for i in range(len(field)):
        if (field[i] == -1):
            input_vector[10 * i + 9] = 1
        else:
            if (min_element == 1e9):
                print(field, min_element)
            input_vector[10 * i + min(8, field[i] - min_element)] = 1
    if (next > 0):
        input_vector[180 + min(8, next - min_element)] = 1
    else:
        input_vector[180 + 9 - next] = 1
    input_vector[193], input_vector[194], input_vector[195] = plus, minus, op

    output_vector = np.zeros(20)
    if (op):
        output_vector[18 + decision] = 1
    else:
        output_vector[decision] = 1
    
    return input_vector, output_vector
    
x = []
y = []

def read_data():
    fin = open('decisions.txt', 'r')
    while True:
        # try:
        if True:
            field = list(map(int, fin.readline().split()))
            plus, minus = map(int, fin.readline().split())
            next = int(fin.readline())
            op = int(fin.readline())
            decision = int(fin.readline())
            fin.readline()
            input_vector, output_vector = preprocess_data(field, plus, minus, next, op, decision)
            x.append(input_vector)
            y.append(output_vector)
        # except:
        #     break

read_data()
print(len(x))


[-1, -1, -1, -2, -1] 1000000000.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices